# Getting started

Once you've chosen your scenario from the README, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

## Scenario and Problem Statement

In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.__version__

u'0.19.0'

In [2]:
liq = pd.read_csv("https://data.iowa.gov/api/views/m3tr-qhgy/rows.csv")https://data.iowa.gov/api/views/m3tr-qhgy/rows.csv

error: [Errno 60] Operation timed out

In [ ]:
liq = pd.read_csv('assets/Iowa_Liquor_sales.csv', dtype = {'Store Number': str, 'Zip Code': str, 'Category': str, 'Vendor Number': str, 'Item Number': str}, parse_dates = ['Date'])

In [ ]:
liq.columns = liq.columns.str.lower().str.replace(' ', '_').str.replace('(', "").str.replace(')', "")
list(liq.columns)
liq.drop(['invoice/item_number',
 'store_number',
 'store_name',
 'address',
 'city',
 'zip_code',
 'store_location',
 'county_number',
 'county',
 'category',
 'category_name',
 'vendor_number',
 'vendor_name',
 'item_number',
 'item_description',
 'pack',
 'bottle_volume_ml',
 'state_bottle_cost',
 'state_bottle_retail',
 'bottles_sold',
 'volume_sold_liters',
 'volume_sold_gallons'], axis=1, inplace=True)

In [ ]:
liq.dtypes

In [ ]:
#do some type converstions
liq.date = pd.to_datetime(liq.date)
# liq['state_bottle_cost_usd'] = liq.state_bottle_cost.str.strip('$').astype(float)
# liq['state_bottle_retail_usd'] = liq.state_bottle_retail.str.strip('$').astype(float)
liq['sale_usd'] = liq.sale_dollars.str.strip('$').astype(float)
# liq.drop(['state_bottle_cost', 'state_bottle_retail', 'sale_dollars'], axis=1, inplace = True)
# liq.dtypes
# decided to keep county number as a float because some stores have missing 
# values for county and I want to be able to just do "np.mean" to account for that (will skip the NAs)

In [ ]:
#now I want to create a series of dummy columns so I can have both the total and by-month values I want.
liq['month'] = liq.date.apply(lambda d: d.month)

In [ ]:
liq = pd.concat([liq, pd.get_dummies(liq.month, prefix='month')], axis=1)
month_cats = list(pd.get_dummies(liq.month, prefix='month').columns) #just in case I want to refer to all of the months later.

In [ ]:
#Now I want to separate my dataframe into 2015 and 2016 so I can use 2015 to make the model and 2016 to test it.
liq['year'] = liq['date'].apply(lambda d: d.year).astype(str)
liq[['year', 'date']].head()

In [ ]:
liq2015 = liq[liq.year=='2015']

In [ ]:
liq2016 = liq[liq.year=='2016']

In [ ]:
liq_jan2015 = liq2015[liq2015.month == 1]

In [ ]:
liq_feb2015 = liq2015[liq2015.month == 2]

In [ ]:
liq_mar2015 = liq2015[liq2015.month == 3]

In [ ]:
jan2015 = liq_jan2015[['county','sale_usd']].groupby(by='county', as_index=False).aggregate(sum)

In [ ]:
feb2015 = liq_feb2015[['county','sale_usd']].groupby(by='county', as_index=False).aggregate(sum)

In [ ]:
mar2015 = liq_mar2015[['county','sale_usd']].groupby(by='county', as_index=False).aggregate(sum)

In [ ]:
all2015 = liq2015[['county','sale_usd']].groupby(by='county', as_index=False).aggregate(sum)

In [ ]:
merged2015 = pd.merge(all2015, jan2015, how='inner', on='county', suffixes=['_all2015', 'b'])
merged2015 = pd.merge(merged2015, feb2015, how='inner', on='county', suffixes=['c', 'd'])
merged2015 = pd.merge(merged2015, mar2015, how='inner', on='county', suffixes=['e', 'f'])
merged2015.head()

In [ ]:
import statsmodels.api as sm
import patsy

In [ ]:
y, X = patsy.dmatrices('sale_usda ~ sale_usdb + sale_usde + sale_usdf', data=merged)
sm.OLS(y, X).fit().summary()

In [ ]:
predictions = sm.OLS(y, X).fit().predict()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from scipy import stats

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(y, predictions)
stats.pearsonr(merged['sale_usda'], predictions)

In [ ]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics,linear_model

lm = linear_model.LinearRegression()
model = lm.fit(merged.ix[:,'sale_usdb': 'sale_usdf'], merged['sale_usda'])
# Perform 6-fold cross validation

scores = cross_val_score(model, merged.ix[:,'sale_usdb': 'sale_usdf'], merged['sale_usda'], cv=3)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(model, merged.ix[:,'sale_usdb': 'sale_usdf'], merged['sale_usda'], cv=3)
plt.scatter(merged['sale_usda'], predictions)
accuracy = metrics.r2_score(merged['sale_usda'], predictions)
print "Cross-Predicted Accuracy:", accuracy